In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from conv_ae_3d.metrics import dice_coefficient
from src.plotting_utils import write_isosurface_plot_from_arr
from src.evaluation.extract_num_components import compute_num_components, compute_surface_area_volume_ratio

In [ ]:
# Conv AEs

In [ ]:
run_base_dir = Path('/Volumes/My Passport for Mac/mf-ae/mf-ae-output')
run_dirs = [run_base_dir / f'eval_v4_{i}' for i in range(3)]

list(map(lambda x: x.exists(), run_dirs))

In [ ]:
def plot_metrics_vs_latent_dim(run_dirs, epoch, xs, metrics):
    fig, axs = plt.subplots(1, len(metrics), figsize=(3*len(metrics), 3), dpi=200)
    
    xs = 1.0 / (np.array(xs) / 256**3)
    
    for i, metric in enumerate(metrics):
        
        ys = []
        ymins_95 = []
        ymaxs_95 = []
    
        for run_dir in run_dirs:
            metric_path = run_dir / 'metrics' / f'val_metrics_{epoch}.csv'
            metric_df = pd.read_csv(metric_path)
            ys.append(metric_df.mean()[metric])
            ymins_95.append(np.quantile(metric_df[metric], 0.05))
            ymaxs_95.append(np.quantile(metric_df[metric], 0.95))
            
        axs[i].plot(xs, ys, 'o-', label='mean')
        axs[i].fill_between(xs, ymins_95, ymaxs_95, alpha=0.3, label='95% CI')
        axs[i].set_xlabel('Compression ratio')
        axs[i].set_title(metric)
        axs[i].set_xscale('log')
        
    axs[0].legend()
        
    fig.tight_layout()
    plt.show()
    


In [ ]:
def plot_losses(run_dirs, bottleneck_sizes):
    fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)
    
    for run_dir, bs in zip(run_dirs, bottleneck_sizes):
        loss_path = run_dir / 'loss_history.json'
        loss_df = pd.read_json(loss_path)
        ax.plot(loss_df.index, loss_df.iloc[:, 0], label=f'{bs}')
        
    ax.legend()
    ax.set_yscale('log')
    ax.set_xlabel('epoch')
    ax.set_ylabel('loss')
    fig.tight_layout()
    plt.show()
    

In [ ]:
# TODO - write out isosurface plots for v3_3 data next

def write_isosurface_plots(run_dir):
    results_folder = run_dir / 'final_val_predictions'
    filepaths = list(results_folder.glob('*.npz'))
    
    assert len(filepaths) > 0, f"No files found in {results_folder}"
    
    for filepath in filepaths[:25]:
        data = np.load(filepath)
        data, pred = data['data'], data['pred']
        
        isosurface_folder = results_folder / "isosurface_plots"
        isosurface_folder.mkdir(exist_ok=True)
        
        write_isosurface_plot_from_arr(data,
                                       outname=isosurface_folder / f"{filepath.stem}_original.png",
                                       level=0.5,
                                       dx=256,
                                       verbose=True)
        
        write_isosurface_plot_from_arr(pred,
                                       outname=isosurface_folder / f"{filepath.stem}_reconstructed.png",
                                       level=0.5,
                                        dx=256,
                                       verbose=True)


In [ ]:
def plot_metric_vs_num_components(run_dir):
    # For each metric, on each validation case, extract the number of components and the metric value, then plot average metric value vs number of components
    # Only plot a single metric
    # run_dir has the results for a single bottleneck size
    
    results_folder = run_dir / 'final_val_predictions'
    filepaths = list(results_folder.glob('*.npz'))
    num_components = []
    sigmas = []
    dice_vals = []

    assert len(filepaths) > 0, f"No files found in {results_folder}"

    for filepath in filepaths:
        data = np.load(filepath)
        data, pred = data['data'], data['pred']
        
        num_components.append(compute_num_components(data))
        sigmas.append(compute_surface_area_volume_ratio(data))
        dice_vals.append(dice_coefficient(data, pred))
        
        
    fig, axs = plt.subplots(1, 2, figsize=(6, 3), dpi=200)
    
    ax = axs[0]
    ax.set_xlabel('Num components')
    ax.set_ylabel('Dice')
    ax.scatter(num_components, dice_vals)
    
    ax = axs[1]
    ax.set_xlabel('Surface area to volume ratio')
    ax.set_ylabel('Dice coefficient')
    ax.scatter(sigmas, dice_vals, alpha=0.5)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    fig.tight_layout()
    plt.show()




In [ ]:
# Use latex for plots
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern"],
    "font.size": 12
})

# Test plot

plt.plot([1, 2, 3], [1, 2, 3])
plt.xlabel(r"$\alpha$")
plt.title(r"$\beta$")
plt.show()

In [ ]:

bottleneck_sizes = [131000, 32768, 8192]
plot_metrics_vs_latent_dim(run_dirs, 50, bottleneck_sizes, ['MSE', 'MAE', 'LINF', 'DICE', 'HAUSDORFF'])
plot_metrics_vs_latent_dim(run_dirs, 50, bottleneck_sizes, ['DICE'])



In [ ]:
# Run for all conv AEs

for run_dir in run_dirs:
    #write_isosurface_plots(run_dir)
    plot_metric_vs_num_components(run_dir)


In [ ]:
# Now repeat all plots for linear bottlenecks

run_base_dir = Path('/Users/murray/Projects/multphase_flow_encoder/multiphase_flow_encoder/output/lassen/mf-ae-output')
run_dirs = [run_base_dir / f'eval_v4_{i}' for i in range(3, 6)]

list(map(lambda x: x.exists(), run_dirs))

In [ ]:
bottleneck_sizes = [800, 400, 100]
plot_metrics_vs_latent_dim(run_dirs, 50, bottleneck_sizes, ['MSE', 'MAE', 'LINF', 'DICE', 'HAUSDORFF'])

for run_dir in run_dirs:
    write_isosurface_plots(run_dir)
    plot_metric_vs_num_components(run_dir)